In [1]:
import ollama
import os
import pdfplumber
import pandas as pd
import re
import json
from pathlib import Path
from typing import List, Dict, Optional
import pytesseract
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
import os

In [2]:
import os

# Windows example – adjust the path to where tesseract is installed
os.environ["PATH"] += os.pathsep + r"C:\Program Files\Tesseract-OCR"

# Now test again
import pytesseract
print(pytesseract.get_tesseract_version())

5.5.0.20241111


In [9]:
SELECTED_PDF = '1069_01-01-2025 tot 01-01-2027_akkoord.pdf'

In [10]:
def lees_pdf_ocr(pdf_path: str) -> str:
    """PDF lezen met OCR voor gescande documenten"""
    try:
        import fitz  # PyMuPDF
        from PIL import Image
        import pytesseract
        import io
        
        tekst = ""
        pdf_document = fitz.open(pdf_path)
        print(f"📄 PDF heeft {len(pdf_document)} pagina's (OCR modus)")
        
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            
            # Probeer eerst normale tekst extractie
            page_text = page.get_text()
            
            if page_text.strip():
                # Als er tekst is, gebruik die
                tekst += page_text + "\n\n"
                print(f"   Pagina {page_num + 1}: Tekst gevonden ({len(page_text)} karakters)")
            else:
                # Geen tekst gevonden, gebruik OCR
                print(f"   Pagina {page_num + 1}: Geen tekst - OCR gebruiken...")
                
                # Converteer pagina naar afbeelding
                mat = fitz.Matrix(2, 2)  # 2x zoom voor betere OCR
                pix = page.get_pixmap(matrix=mat)
                img_data = pix.tobytes("png")
                
                # PIL Image voor OCR
                image = Image.open(io.BytesIO(img_data))
                
                # OCR met Tesseract
                ocr_text = pytesseract.image_to_string(image, lang='nld+eng')
                
                if ocr_text.strip():
                    tekst += ocr_text + "\n\n"
                    print(f"   Pagina {page_num + 1}: OCR tekst ({len(ocr_text)} karakters)")
                else:
                    print(f"   Pagina {page_num + 1}: Geen tekst gevonden met OCR")
        
        pdf_document.close()
        print(f"✅ PDF met OCR gelezen: {len(tekst)} totaal karakters")
        return tekst
        
    except ImportError as e:
        print(f"❌ Ontbrekende dependency: {e}")
        print("💡 Installeer: pip install PyMuPDF pillow pytesseract")
        return lees_pdf_advanced(pdf_path)  # Fallback naar normale PDF reader
    except Exception as e:
        print(f"❌ Fout bij OCR PDF extractie: {e}")
        return lees_pdf_advanced(pdf_path)  # Fallback naar normale PDF reader

def lees_pdf_smart(pdf_path: str) -> str:
    """Slimme PDF reader die automatisch kiest tussen normale en OCR extractie"""
    try:
        # Probeer eerst normale extractie
        normale_tekst = lees_pdf(pdf_path)
        
        # Check of er genoeg tekst is gevonden
        if len(normale_tekst.strip()) > 100:  # Als meer dan 100 karakters
            print("📖 Normale PDF extractie succesvol")
            return normale_tekst
        else:
            print("📖 Weinig tekst gevonden - probeer OCR...")
            return lees_pdf_ocr(pdf_path)
            
    except Exception as e:
        print(f"❌ Fout bij slimme PDF extractie: {e}")
        return ""

# Voeg toe aan je bestaande CEL 4 sectie:
print("🔍 OCR PDF functies toegevoegd!")
print("   - lees_pdf_ocr(): Voor gescande PDF's")
print("   - lees_pdf_smart(): Automatische keuze tussen normale/OCR extractie")

🔍 OCR PDF functies toegevoegd!
   - lees_pdf_ocr(): Voor gescande PDF's
   - lees_pdf_smart(): Automatische keuze tussen normale/OCR extractie


In [16]:
tekst = lees_pdf_ocr(SELECTED_PDF)
print(tekst)

📄 PDF heeft 8 pagina's (OCR modus)
   Pagina 1: Geen tekst - OCR gebruiken...
   Pagina 1: OCR tekst (1535 karakters)
   Pagina 2: Geen tekst - OCR gebruiken...
   Pagina 2: OCR tekst (2194 karakters)
   Pagina 3: Geen tekst - OCR gebruiken...
   Pagina 3: OCR tekst (509 karakters)
   Pagina 4: Geen tekst - OCR gebruiken...
   Pagina 4: OCR tekst (2701 karakters)
   Pagina 5: Geen tekst - OCR gebruiken...
   Pagina 5: OCR tekst (2640 karakters)
   Pagina 6: Geen tekst - OCR gebruiken...
   Pagina 6: OCR tekst (1264 karakters)
   Pagina 7: Geen tekst - OCR gebruiken...
   Pagina 7: OCR tekst (2910 karakters)
   Pagina 8: Geen tekst - OCR gebruiken...
   Pagina 8: OCR tekst (990 karakters)
✅ PDF met OCR gelezen: 14759 totaal karakters
JADM U on

ONDERHANDELINGSRESULTAAT CAO ADM EUROPOORT

1 januari 2025 tot en met 31 december 2026

1. Looptijd
De cao heeft een looptijd van 1 januari 2025 tot en met 31 december 2026.

2. Loon

De salarissen worden gedurende de looptijd van de cao worden a

In [11]:
# CEL 4: PDF LEES FUNCTIES
print("📖 PDF LEES FUNCTIES DEFINIËREN")
print("=" * 50)

def lees_pdf(pdf_path: str) -> str:
    """Standaard PDF lezen met pdfplumber"""
    try:
        tekst = ""
        with pdfplumber.open(pdf_path) as pdf:
            print(f"📄 PDF heeft {len(pdf.pages)} pagina's")
            
            for i, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                if page_text:
                    tekst += page_text + "\n\n"
                    print(f"   Pagina {i}: {len(page_text)} karakters")
                else:
                    print(f"   Pagina {i}: geen tekst gevonden")
                
        print(f"✅ PDF gelezen: {len(tekst)} totaal karakters")
        return tekst
    except Exception as e:
        print(f"❌ Fout bij lezen PDF: {e}")
        return ""

def lees_pdf_advanced(pdf_path: str) -> str:
    """Geavanceerde PDF lezen met tabellen"""
    try:
        tekst = ""
        with pdfplumber.open(pdf_path) as pdf:
            print(f"📄 PDF heeft {len(pdf.pages)} pagina's")
            
            for i, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                tables = page.extract_tables()
                
                if page_text:
                    tekst += page_text + "\n"
                
                if tables:
                    for table in tables:
                        for row in table:
                            if row:
                                row_text = " | ".join([cell or "" for cell in row])
                                tekst += row_text + "\n"
                    print(f"   Pagina {i}: {len(tables)} tabel(len) gevonden")
                
                tekst += "\n"
                
        print(f"✅ Geavanceerde PDF extractie: {len(tekst)} karakters")
        return tekst
    except Exception as e:
        print(f"❌ Fout bij geavanceerde PDF extractie: {e}")
        return lees_pdf(pdf_path)

print("✅ PDF functies gedefinieerd!")

📖 PDF LEES FUNCTIES DEFINIËREN
✅ PDF functies gedefinieerd!


In [14]:
# CEL 8: REGEX FUNCTIE VOOR LOON ZINNEN
print("🔍 REGEX FUNCTIE VOOR LOON ZINNEN DEFINIËREN")
print("=" * 50)

def vind_loon_zinnen(tekst: str) -> List[str]:
    """Vindt zinnen die mogelijk over loonsverhogingen gaan"""
    
    # Regex patterns voor loonsverhogingen
    percentage_patterns = [
        r'[0-9]+[,.]?[0-9]*\s*%',           # 5%, 3.5%, 2,5%
        r'[0-9]+[,.]?[0-9]*\s*procent',     # 5 procent, 3,5 procent
        r'[0-9]+[,.]?[0-9]*\s*pct',         # 5 pct
    ]
    
    # Loon-gerelateerde woorden
    loon_woorden = [
        'loon', 'salaris', 'bezoldiging', 'vergoeding', 'beloning',
        'verhog', 'toesla', 'indexa', 'aanpass', 'structureel',
        'cao', 'arbeidsvoorwaarden'
    ]
    
    # Split tekst in zinnen
    zinnen = re.split(r'[.!?]+', tekst)
    
    relevante_zinnen = []
    
    for zin in zinnen:
        zin = zin.strip()
        if len(zin) < 20:  # Skip te korte zinnen
            continue
            
        # Check of zin percentage bevat
        heeft_percentage = any(re.search(pattern, zin, re.IGNORECASE) 
                              for pattern in percentage_patterns)
        
        # Check of zin loon-woorden bevat
        heeft_loon_woord = any(woord.lower() in zin.lower() 
                              for woord in loon_woorden)
        
        if heeft_percentage and heeft_loon_woord:
            relevante_zinnen.append(zin)
    
    print(f"✅ {len(relevante_zinnen)} relevante zinnen gevonden")
    return relevante_zinnen

print("✅ Regex functie gedefinieerd!")

🔍 REGEX FUNCTIE VOOR LOON ZINNEN DEFINIËREN
✅ Regex functie gedefinieerd!


In [17]:
# CEL 9: LOON ZINNEN ZOEKEN IN PDF TEKST
print("🔍 LOON ZINNEN ZOEKEN IN PDF TEKST")
print("=" * 50)

if tekst:
    print(f"📄 Doorzoeken van {len(tekst)} karakters tekst...")
    
    # Zoek relevante zinnen
    loon_zinnen = vind_loon_zinnen(tekst)
    
    if loon_zinnen:
        print(f"\n📋 GEVONDEN LOON ZINNEN ({len(loon_zinnen)}):")
        print("=" * 50)
        
        for i, zin in enumerate(loon_zinnen, 1):
            print(f"{i}. {zin}")
            print("-" * 50)
            
        # Bewaar voor volgende stap
        GEVONDEN_ZINNEN = loon_zinnen
        print("✅ Zinnen opgeslagen voor AI analyse!")
    else:
        print("❌ Geen relevante loon zinnen gevonden")
        print("💡 Probeer de geavanceerde PDF extractie of controleer je PDF inhoud")
        GEVONDEN_ZINNEN = []
else:
    print("❌ Geen PDF tekst beschikbaar")
    GEVONDEN_ZINNEN = []

🔍 LOON ZINNEN ZOEKEN IN PDF TEKST
📄 Doorzoeken van 14759 karakters tekst...
✅ 1 relevante zinnen gevonden

📋 GEVONDEN LOON ZINNEN (1):
1. Loon

De salarissen worden gedurende de looptijd van de cao worden als volgt verhoogd:

9% met ingang van 1 januari 2025
4% met ingang van 1 januari 2026

3
--------------------------------------------------
✅ Zinnen opgeslagen voor AI analyse!


In [21]:
def extraheer_specifieke_loonsverhogingen(zinnen: List[str], model_name: str = "gemma3:latest") -> List[Dict]:
    """
    Verbeterde AI-functie die alleen werkelijke loonsverhogingen extraheert,
    inclusief meerdere percentages/datums binnen één zin
    """
    
    prompt_template = """Je bent een expert in het analyseren van Nederlandse CAO-teksten met focus op concrete, definitieve loonafspraken. 
Je krijgt één zin als input. Volg strikt de regels:

1. **Formaatvereisten**  
   - De zin **moet** een percentage bevatten (een getal direct gevolgd door '%', bijv. "3,5%" of "7%").  
   - De zin **moet** een datum bevatten in het formaat 'DD-MM-YYYY', 'DD/MM/YYYY', 'YYYY-MM-DD', of 'per DD maand YYYY'.  
   - Zinnen zonder percentage of zonder datum classificeer je direct als "Geen Loonstijging".

2. **Percentage-controle**  
   - Als het percentage **groter is dan 10%**, beschouw dit als "Geen Loonstijging".  
   - Alleen percentages ≤ 10% kunnen een "Loonstijging" zijn, mits aan alle andere voorwaarden voldaan is.

3. **Meerdere percentages/datums - ZEER BELANGRIJK**
   - Scan de VOLLEDIGE zin grondig naar ALLE percentages en ALLE datums.
   - Als de zin meerdere percentages EN meerdere datums bevat, geef dan ALLE mogelijke combinaties terug.
   - Bullet-point structuren zoals "• per 1 januari 2025: 4% • per 1 januari 2026: 1,5%" moeten volledig worden geëxtraheerd.
   - Elke geldige percentage-datum combinatie moet een apart item in de "verhogingen" array worden.
   - Zorg ervoor dat je GEEN verhogingen mist, ook niet bij complexe zinnen met veel informatie.

4. **Datumformaten die je moet herkennen**
   - "per 1 januari 2025", "per 1 september 2026"
   - "1-1-2025", "01/01/2025", "2025-01-01"
   - "1 januari 2025", "15 maart 2026"

5. **Uitsluitingen**  
   - Negeer (classificeer als "Geen Loonstijging") alle zinnen die beginnen met of de termen bevatten:  
     "Rekenvoorbeeld", "Voorbeeld:", "Stel dat", "indicatief", "globaal", "ambitie", "berekening", "was het minimum", "bedraagt minimaal".  
   - Historische gegevens (2024 of eerder) zijn geen toekomstige CAO-afspraken.
   - Algemene doelstellingen of rekenvoorbeelden tellen niet als definitieve afspraken.

6. **JSON-uitvoer**  
   Geef ALTIJD een JSON-object terug met exact deze structuur:
   ```json
   {{
     "classificatie": "Loonstijging" | "Geen Loonstijging",
     "verhogingen": [
       {{
         "percentage": "<percentage>",
         "datum": "YYYY-MM-DD"
       }}
     ],
     "uitleg": "<korte toelichting>"
   }}
   ```
   
   Voorbeeld voor meerdere verhogingen uit jouw voorbeeldzin:
   ```json
   {{
     "classificatie": "Loonstijging",
     "verhogingen": [
       {{
         "percentage": "4%",
         "datum": "2025-01-01"
       }},
       {{
         "percentage": "1,5%",
         "datum": "2026-01-01"
       }},
       {{
         "percentage": "1,5%",
         "datum": "2026-09-01"
       }}
     ],
     "uitleg": "Drie loonsverhogingen gevonden in bullet-point structuur"
   }}
   ```

Zin om te analyseren: "{zin}"
"""

    resultaten = []
    
    print(f"🎯 Start SPECIFIEKE loonsverhogingen extractie met {model_name}")
    
    for i, zin in enumerate(zinnen, 1):
        print(f"   Analyseren {i}/{len(zinnen)}: {zin[:80]}...")
        
        try:
            volledige_prompt = prompt_template.format(zin=zin)
            
            response = ollama.chat(
                model=model_name,
                messages=[{'role': 'user', 'content': volledige_prompt}],
                options={'temperature': 0.05, 'top_p': 0.8}
            )
            
            output_text = response['message']['content']
            
            # Parse JSON
            json_start = output_text.find('{')
            json_end = output_text.rfind('}') + 1
            
            if json_start != -1 and json_end > json_start:
                json_string = output_text[json_start:json_end]
                data = json.loads(json_string)
                
                # Controleer of het een loonstijging is
                if data.get('classificatie') == 'Loonstijging' and data.get('verhogingen'):
                    verhogingen_lijst = data.get('verhogingen', [])
                    
                    # Verwerk elke verhoging afzonderlijk
                    for verhoging in verhogingen_lijst:
                        percentage = verhoging.get('percentage')
                        datum = verhoging.get('datum')
                        
                        if percentage and datum:
                            # Extra validatie: percentage moet realistisch zijn (0.5% - 10%)
                            try:
                                # Verwerk Nederlandse komma-notatie (3,75% -> 3.75)
                                perc_str = percentage.replace('%', '').replace(',', '.')
                                perc_value = float(perc_str)
                                if 0.5 <= perc_value <= 10:  # Strenger: max 8% voor realistische CAO-verhogingen
                                    individueel_resultaat = {
                                        'classificatie': 'Loonstijging',
                                        'percentage': percentage,
                                        'datum': datum,
                                        'uitleg': data.get('uitleg', ''),
                                        'originele_zin': zin
                                    }
                                    resultaten.append(individueel_resultaat)
                                    print(f"   ✅ RELEVANTE VERHOGING: {percentage} op {datum}")
                                else:
                                    print(f"   ❌ Percentage {perc_value}% buiten realistische CAO-range (0.5-8%)")
                            except ValueError:
                                print(f"   ❌ Ongeldig percentage formaat: '{percentage}'")
                        else:
                            print(f"   ❌ Incomplete verhoging: percentage='{percentage}', datum='{datum}'")
                else:
                    print(f"   ❌ Geen loonstijging: {data.get('classificatie', 'onbekend')}")
                    
            else:
                print(f"   ⚠️  Geen geldige JSON ontvangen")
                
        except Exception as e:
            print(f"   ❌ Fout bij verwerken: {e}")
    
    print(f"✅ SPECIFIEKE extractie voltooid: {len(resultaten)} definitieve loonsverhogingen")
    return resultaten

def detecteer_meerdere_percentages_regex(zin: str) -> List[Dict]:
    """
    Backup functie die regex gebruikt om meerdere percentages en datums te vinden
    als de AI het mist. Gespecialiseerd voor CAO bullet-point structuren.
    """
    import re
    
    # Verbeterde regex patronen voor CAO-structuren
    percentage_pattern = r'\b\d{1,2}(?:[,.]\d+)?\s*%'
    
    # Uitgebreide datum patronen voor Nederlandse CAO's
    datum_patterns = [
        r'\b\d{1,2}[-/]\d{1,2}[-/]\d{4}\b',  # DD-MM-YYYY of DD/MM/YYYY
        r'\b\d{4}[-/]\d{1,2}[-/]\d{1,2}\b',  # YYYY-MM-DD of YYYY/MM/DD
        r'\b\d{1,2}\s+(januari|februari|maart|april|mei|juni|juli|augustus|september|oktober|november|december)\s+\d{4}\b',  # DD maand YYYY
        r'\bper\s+\d{1,2}\s+(januari|februari|maart|april|mei|juni|juli|augustus|september|oktober|november|december)\s+\d{4}\b',  # per DD maand YYYY
        r'\bper\s+\d{1,2}[-/]\d{1,2}[-/]\d{4}\b',  # per DD-MM-YYYY
        r'\bper\s+\d{4}[-/]\d{1,2}[-/]\d{1,2}\b',  # per YYYY-MM-DD
    ]
    
    # Speciale detectie voor bullet-point structuren
    bullet_pattern = r'•\s*per\s+(\d{1,2}\s+\w+\s+\d{4}|\d{1,2}[-/]\d{1,2}[-/]\d{4}|\d{4}[-/]\d{1,2}[-/]\d{1,2}):\s*(\d{1,2}(?:[,.]\d+)?\s*%)'
    
    resultaten = []
    
    # Probeer eerst bullet-point detectie
    bullet_matches = re.findall(bullet_pattern, zin, re.IGNORECASE)
    if bullet_matches:
        print(f"   🔍 REGEX BULLET: Gevonden {len(bullet_matches)} bullet-point verhogingen")
        for i, (datum, percentage) in enumerate(bullet_matches):
            # Normaliseer datum formaat
            datum_normalized = normaliseer_datum(datum.strip())
            
            resultaat = {
                'classificatie': 'Loonstijging',
                'percentage': percentage.strip(),
                'datum': datum_normalized,
                'uitleg': f'Bullet-point detectie - item {i+1}',
                'originele_zin': zin
            }
            resultaten.append(resultaat)
    else:
        # Fallback naar algemene detectie
        percentages = re.findall(percentage_pattern, zin, re.IGNORECASE)
        
        # Vind alle datums
        datums = []
        for pattern in datum_patterns:
            matches = re.findall(pattern, zin, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):  # Voor patronen met groepen
                    datums.append(' '.join(match).strip())
                else:
                    datums.append(match.strip())
        
        # Verwijder duplicaten en sorteer
        datums = list(dict.fromkeys(datums))  # Behoudt volgorde, verwijdert duplicaten
        
        # Combineer percentages met datums
        if len(percentages) > 1 and len(datums) >= len(percentages):
            print(f"   🔍 REGEX ALGEMEEN: Gevonden {len(percentages)} percentages en {len(datums)} datums")
            for i, percentage in enumerate(percentages):
                if i < len(datums):
                    datum_normalized = normaliseer_datum(datums[i])
                    
                    resultaat = {
                        'classificatie': 'Loonstijging',
                        'percentage': percentage.strip(),
                        'datum': datum_normalized,
                        'uitleg': f'Algemene regex detectie - combinatie {i+1}',
                        'originele_zin': zin
                    }
                    resultaten.append(resultaat)
    
    return resultaten

def normaliseer_datum(datum_str: str) -> str:
    """
    Normaliseert verschillende datumformaten naar YYYY-MM-DD
    """
    import re
    from datetime import datetime
    
    # Verwijder 'per' en extra whitespace
    datum_str = re.sub(r'\bper\s+', '', datum_str, flags=re.IGNORECASE).strip()
    
    # Nederlandse maanden mapping
    maanden = {
        'januari': '01', 'februari': '02', 'maart': '03', 'april': '04',
        'mei': '05', 'juni': '06', 'juli': '07', 'augustus': '08',
        'september': '09', 'oktober': '10', 'november': '11', 'december': '12'
    }
    
    try:
        # DD maand YYYY formaat
        match = re.match(r'(\d{1,2})\s+(\w+)\s+(\d{4})', datum_str, re.IGNORECASE)
        if match:
            dag, maand_naam, jaar = match.groups()
            maand_naam_lower = maand_naam.lower()
            if maand_naam_lower in maanden:
                return f"{jaar}-{maanden[maand_naam_lower]}-{dag.zfill(2)}"
        
        # DD-MM-YYYY of DD/MM/YYYY formaat
        match = re.match(r'(\d{1,2})[-/](\d{1,2})[-/](\d{4})', datum_str)
        if match:
            dag, maand, jaar = match.groups()
            return f"{jaar}-{maand.zfill(2)}-{dag.zfill(2)}"
        
        # YYYY-MM-DD of YYYY/MM/DD formaat (al correct of bijna correct)
        match = re.match(r'(\d{4})[-/](\d{1,2})[-/](\d{1,2})', datum_str)
        if match:
            jaar, maand, dag = match.groups()
            return f"{jaar}-{maand.zfill(2)}-{dag.zfill(2)}"
            
    except Exception as e:
        print(f"   ⚠️ Datum normalisatie fout voor '{datum_str}': {e}")
    
    # Als normalisatie faalt, return originele string
    return datum_str

def filter_en_sorteer_resultaten(resultaten: List[Dict]) -> List[Dict]:
    """
    Extra filtering en sortering van de AI-resultaten
    """
    print("\n🔍 EXTRA FILTERING VAN RESULTATEN")
    print("=" * 50)
    
    gefilterde_resultaten = []
    
    for resultaat in resultaten:
        zin = resultaat.get('originele_zin', '').lower()
        percentage = resultaat.get('percentage', '')
        datum = resultaat.get('datum', '')
        
        # Skip voorbeelden en rekenvoorbeelden
        skip_termen = ['rekenvoorbeeld', 'voorbeeld:', 'stel dat', 'sprake', 'indicatief', 'ambitie', 'bijvoorbeeld', 
               'berekening', 'was het minimum', 'bedraagt minimaal', 'kan oplopen',
               'hypothetisch', 'fictief', 'simulatie', 'scenario', 'indien',
               'ter illustratie', 'mogelijke', 'potentiële', 'geschatte']
        if any(term in zin for term in skip_termen):
            print(f"   ❌ Overgeslagen (voorbeeld/berekening): {percentage} - {datum}")
            continue
            
        # Skip historische data (2024 en eerder)
        if datum and '2024' in datum:
            print(f"   ❌ Overgeslagen (historische data): {percentage} - {datum}")
            continue
            
        # Skip zeer hoge percentages (waarschijnlijk geen standaard loonsverhogingen)
        try:
            # Verwerk Nederlandse komma-notatie
            perc_str = percentage.replace('%', '').replace(',', '.')
            perc_num = float(perc_str)
            if perc_num > 10:  # Strenger: CAO-verhogingen zijn meestal 1-8%
                print(f"   ❌ Overgeslagen (te hoog voor CAO): {percentage} - {datum}")
                continue
        except ValueError:
            print(f"   ❌ Ongeldig percentage bij filtering: '{percentage}'")
            continue
            
        # Skip duplicaten op basis van datum + percentage
        sleutel = f"{datum}_{percentage}"
        if not any(f"{r.get('datum')}_{r.get('percentage')}" == sleutel for r in gefilterde_resultaten):
            gefilterde_resultaten.append(resultaat)
            print(f"   ✅ Behouden: {percentage}% op {datum}")
        else:
            print(f"   ❌ Overgeslagen (duplicaat): {percentage}% - {datum}")
    
    # Sorteer op datum
    try:
        gefilterde_resultaten.sort(key=lambda x: x.get('datum', ''))
    except:
        pass
    
    print(f"\n📊 FINALE FILTERING: {len(gefilterde_resultaten)} definitieve loonsverhogingen")
    return gefilterde_resultaten

def controleer_meerdere_verhogingen_per_zin(zinnen: List[str]) -> List[str]:
    """
    Controleer welke zinnen mogelijk meerdere percentages/datums bevatten
    """
    import re
    
    verdachte_zinnen = []
    
    for zin in zinnen:
        # Tel percentages (verbeterde regex)
        percentages = len(re.findall(r'\b\d{1,2}(?:[,.]\d+)?\s*%', zin))
        
        # Tel datums (uitgebreide patronen voor CAO's)
        datum_patronen = [
            r'\b\d{1,2}[-/]\d{1,2}[-/]\d{4}\b',  # DD-MM-YYYY
            r'\b\d{4}[-/]\d{1,2}[-/]\d{1,2}\b',  # YYYY-MM-DD
            r'\b\d{1,2}\s+(?:januari|februari|maart|april|mei|juni|juli|augustus|september|oktober|november|december)\s+\d{4}\b',  # DD maand YYYY
            r'\bper\s+\d{1,2}[-/]\d{1,2}[-/]\d{4}\b',  # per DD-MM-YYYY
            r'\bper\s+\d{4}[-/]\d{1,2}[-/]\d{1,2}\b',  # per YYYY-MM-DD
            r'\bper\s+\d{1,2}\s+(?:januari|februari|maart|april|mei|juni|juli|augustus|september|oktober|november|december)\s+\d{4}\b',  # per DD maand YYYY
        ]
        
        datums = 0
        for patroon in datum_patronen:
            datums += len(re.findall(patroon, zin, re.IGNORECASE))
        
        # Ook checken op bullet-point structuren
        bullet_items = len(re.findall(r'•\s*per\s+', zin, re.IGNORECASE))
        
        if percentages > 1 or datums > 1 or bullet_items > 1:
            verdachte_zinnen.append(zin)
            print(f"   🔍 Meerdere waarden: {percentages} percentages, {datums} datums, {bullet_items} bullets")
            print(f"      Zin: {zin[:100]}...")
    
    return verdachte_zinnen

# Hoofduitvoering
print("🚀 AI EXTRACTIE UITVOEREN (VERBETERDE VERSIE)")
print("=" * 60)

if 'GEVONDEN_ZINNEN' in globals() and GEVONDEN_ZINNEN:
    print(f"📤 Versturen van {len(GEVONDEN_ZINNEN)} zinnen naar AI...")
    
    # Controleer eerst welke zinnen meerdere verhogingen kunnen bevatten
    print("\n🔍 CONTROLEREN OP MEERDERE VERHOGINGEN PER ZIN:")
    verdachte_zinnen = controleer_meerdere_verhogingen_per_zin(GEVONDEN_ZINNEN)
    if verdachte_zinnen:
        print(f"Gevonden {len(verdachte_zinnen)} zinnen met mogelijk meerdere verhogingen")
    
    SELECTED_MODEL = "gemma3:latest"
    
    # Voer AI extractie uit
    ai_resultaten = extraheer_specifieke_loonsverhogingen(GEVONDEN_ZINNEN, SELECTED_MODEL)
    
    # Voor zinnen die de AI mogelijk gemist heeft, probeer regex backup
    print("\n🔄 REGEX BACKUP CONTROLE:")
    for zin in verdachte_zinnen:
        regex_resultaten = detecteer_meerdere_percentages_regex(zin)
        if regex_resultaten:
            print(f"   ✅ Regex vond {len(regex_resultaten)} extra verhogingen")
            ai_resultaten.extend(regex_resultaten)
    
    if ai_resultaten:
        # Extra filtering
        gefilterde_resultaten = filter_en_sorteer_resultaten(ai_resultaten)
        
        if gefilterde_resultaten:
            print(f"\n📋 DEFINITIEVE LOONSVERHOGINGEN ({len(gefilterde_resultaten)}):")
            print("=" * 60)
            
            for i, resultaat in enumerate(gefilterde_resultaten, 1):
                print(f"{i}. DATUM: {resultaat.get('datum')}")
                print(f"   PERCENTAGE: {resultaat.get('percentage')}")
                print(f"   CONTEXT: {resultaat.get('originele_zin', '')[:120]}...")
                print("-" * 60)
                
            # Bewaar voor verdere analyse
            FINALE_RESULTATEN = gefilterde_resultaten
            print("✅ Verbeterde AI extractie succesvol voltooid!")
            
            # Maak een overzichtstabel
            print(f"\n📈 OVERZICHT LOONSVERHOGINGEN:")
            print("=" * 40)
            for resultaat in gefilterde_resultaten:
                print(f"• {resultaat.get('datum')}: {resultaat.get('percentage')}")
                
        else:
            print("❌ Geen definitieve loonsverhogingen gevonden na filtering")
            FINALE_RESULTATEN = []
    else:
        print("❌ Geen AI resultaten ontvangen")
        FINALE_RESULTATEN = []
else:
    print("❌ Geen zinnen om te verwerken")
    print("💡 Definieer eerst GEVONDEN_ZINNEN variabele")
    FINALE_RESULTATEN = []

🚀 AI EXTRACTIE UITVOEREN (VERBETERDE VERSIE)
📤 Versturen van 1 zinnen naar AI...

🔍 CONTROLEREN OP MEERDERE VERHOGINGEN PER ZIN:
   🔍 Meerdere waarden: 2 percentages, 2 datums, 0 bullets
      Zin: Loon

De salarissen worden gedurende de looptijd van de cao worden als volgt verhoogd:

9% met ingan...
Gevonden 1 zinnen met mogelijk meerdere verhogingen
🎯 Start SPECIFIEKE loonsverhogingen extractie met gemma3:latest
   Analyseren 1/1: Loon

De salarissen worden gedurende de looptijd van de cao worden als volgt ver...
   ✅ RELEVANTE VERHOGING: 9% op 2025-01-01
   ✅ RELEVANTE VERHOGING: 4% op 2026-01-01
✅ SPECIFIEKE extractie voltooid: 2 definitieve loonsverhogingen

🔄 REGEX BACKUP CONTROLE:

🔍 EXTRA FILTERING VAN RESULTATEN
   ✅ Behouden: 9%% op 2025-01-01
   ✅ Behouden: 4%% op 2026-01-01

📊 FINALE FILTERING: 2 definitieve loonsverhogingen

📋 DEFINITIEVE LOONSVERHOGINGEN (2):
1. DATUM: 2025-01-01
   PERCENTAGE: 9%
   CONTEXT: Loon

De salarissen worden gedurende de looptijd van de cao wor